# Introduction
In previous notebooks, we've used the E. coli cor emap to visualize fluxes in metabolism. However there are of course species-differences between *P. thermoglucosidasius* NCBI 11955 and E. coli, so for further analysis of data it would be very benefical to have a map of central carbon metabolism as it is in our strain. This notebook will cover modifying the E. coli core map to fit our wishes as much as possible.

Later we can decide to make one that also includes amino acid metabolism, or any other functionalities we would like.

In [1]:
import cameo
import pandas as pd
import cobra.io
import escher
from escher import Builder
from cobra import Reaction, Metabolite

In [2]:
model = cobra.io.read_sbml_model('../model/g-thermo.xml')

In [3]:
model_e_coli = cameo.load_model('iML1515')

In [4]:
#here I can build the map on the e. coli core backbone
builder = Builder(model=model, map_name = 'e_coli_core.Core metabolism')

In [31]:
#here I can build the map on the e. coli core backbone, taking the initial changes
Builder(model=model, map_json = '../map/g-thermo-map.json')

Builder()

__Glutamate__

I have now observed that we have three glutamate metabolites: L, D and DL glutamate. According to bioinformatic data, our starin can form both L and D glutamate, though L is the predominant form required for biomas for example. 

Here I will remove the DL metabolite and leave the rest. 


In [8]:
#remove ACYSAT, same as CYSAT
model.remove_reactions(model.reactions.ACYSAT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [9]:
model.remove_metabolites(model.metabolites.glu__DL_c)

__Additional changes__ Modifying the map also lets us see some changes that need to be made. 

For example, this strain has been shown to produce formate upon fermentation, and so we need a formate transport reaction that is currently lacking. (data from Tang et al. 2009, doi 10.1002/bit.22181) Bioinformatically, a formate-proton symporter has been identified and so this will be added. 

In [10]:
#add formate transport
model.add_reaction(Reaction(id='FORt'))

In [11]:
model.reactions.FORt.name = 'Transport of formate via proton symport'
model.reactions.FORt.brounds = (-1000,1000)

In [14]:
model.add_metabolites(Metabolite(id='for_e'))

In [28]:
model.metabolites.for_e.name = 'Formate'
model.metabolites.for_e.formula = model.metabolites.for_c.formula
model.metabolites.for_e.name = model.metabolites.for_c.name
model.metabolites.for_e.charge = model.metabolites.for_c.charge
model.metabolites.for_e.compartment = 'e'

In [16]:
model.metabolites.for_e.name = 'Formate'
model.metabolites.h_e.name = 'Proton'
model.metabolites.for_e.formula = model.metabolites.for_c.formula

In [17]:
model.reactions.FORt.add_metabolites({model.metabolites.for_e: -1, model.metabolites.for_c: 1, model.metabolites.h_e:-1, model.metabolites.h_c:1})

add phosphate transport: we supply phosphate in the medium, and it can be taken up by the cell. considering its role in ATP generation it will be an important metabolite to supply in the default medium. 


In [18]:
model.add_metabolites(Metabolite(id='pi_e'))

In [19]:
model.metabolites.pi_e.name = 'Phosphate'
model.metabolites.pi_e.formula = model.metabolites.pi_c.formula
model.metabolites.pi_e.charge = model.metabolites.pi_c.charge
model.metabolites.pi_e.compartment = 'e'
model.metabolites.pi_e.annotation = model_e_coli.metabolites.pi_e.annotation

In [20]:
model.add_reaction(Reaction(id='PIt'))

In [21]:
model.reactions.PIt.name = 'Transport of phosphate'
model.reactions.PIt.bounds = (-1000,1000)

In [22]:
model.reactions.PIt.add_metabolites({model.metabolites.pi_e:-1, model.metabolites.pi_c:1})

In [23]:
model.add_boundary(model.metabolites.pi_e, type = 'exchange', reaction_id = 'EX_pi_e')

Reaction identifier,EX_pi_e
Name,Phosphate exchange
Memory address,0x01602b82dd88
Stoichiometry,pi_e <=> Phosphate <=>
GPR,
Lower bound,-1000.0
Upper bound,1000.0


In [24]:
model.reactions.ICDHhr.id = 'ICDHyr'

In [26]:
#modify name
model.reactions.R5PISO.id = 'RPI'

In [29]:
#save & commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

In looking into the map, I've looked into ATP synthesis, searching for the ATP synthase. In doing so, I ran into ATPPR, which just converts ATP to ADP reversibly. This needs to be removed, and interestingly doesn't even change our biomass prediction. Which is kind of strange, but atleast then I can remove it now.


In [32]:
model.remove_reactions(model.reactions.ATPPT)

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\model.py:716: UserWarning:

need to pass in a list

C:\Users\vivmol\AppData\Local\Continuum\anaconda3\envs\g-thermo\lib\site-packages\cobra\core\group.py:110: UserWarning:

need to pass in a list



In [33]:
#save&commit
cobra.io.write_sbml_model(model,'../model/g-thermo.xml')

__TO DO__:
- the GDH va GDH_nadp node
- Add Biomass rct: which to remove from depiction
- check main exchanges when it is run. 
- add AA metabolism? (or seperate map)

In [ ]:
g_thermo_map = '../databases/Maps/g-thermo-map.json'

In [ ]:
#save map
builder.save_html('example_map.html')